# Patent citation 제작

### 1. patent assignee 제작
- 기존 파일에 옛날 patent 정보 (3,4,5번대) 업데이트 필요

### 2. 기업간 adj matrix 제작
- (patent_id -> cited)를 (gvkey -> gvkey)로 바꾸기

### 3. weight sum
- 전에 구한 weight로 weight sum해서 firm level 변수 제작

### 1. patent assignee 제작
- __6836904번까지 assignee 정보 필요__
- raw data에서 patent_id, raw_assignee 추출  
- raw_assignee -> assignee (assignee_raw join)
- 기존 csv에 합치기 (pd.concat)

__patent_id, raw_assignee 추출__

In [1]:
import os
import csv
import zipfile

os.chdir("E:/patent/all_patent")

In [2]:
zf_patent1 = zipfile.ZipFile('harvard_dataset_final_merged_output_1.zip') 
csv_patent1 = zf_patent1.open('harvard_dataset_final_merged_output_1.csv')
csv_patent_reader1 = csv.reader(csv_patent1)
header = csv_patent_reader1.next()

zf_patent2 = zipfile.ZipFile('harvard_dataset_final_merged_output_2.zip') 
csv_patent2 = zf_patent2.open('harvard_dataset_final_merged_output_2.csv')
csv_patent_reader2 = csv.reader(csv_patent2)
header = csv_patent_reader2.next()

In [3]:
#select the assignee col
assignee_col = []
for i in range(0,len(header)):
    if 'organization' in header[i]:   
        assignee_col.append(i)
        
del assignee_col[-3:]    #delete lawyer org

In [4]:
assignee = []
p_id = []

for line in csv_patent_reader1:
    for i in assignee_col:
        if line[i]=='':
            break
        
        elif line[i]=='nan':
            assignee.append('private')
            p_id.append(line[1])
                        
        else:
            assignee.append(line[i])
            p_id.append(line[1])
                        
for line in csv_patent_reader2:
    if line[i] == '6836905':
        break
    for i in assignee_col:
        if line[i]=='':
            break
        
        elif line[i]=='nan':
            assignee.append('private')
            p_id.append(line[1])
                        
        else:
            assignee.append(line[i])
            p_id.append(line[1])

NameError: name 'pd' is not defined

In [5]:
len(p_id)

6894521

In [9]:
p_id[-1]

'RE45078'

In [10]:
import pandas as pd

df = pd.DataFrame({'patent_id':p_id,'raw_assignee':assignee})
df = df.drop_duplicates()

In [11]:
len(df)

5002108

In [12]:
df.sort_values(by='patent_id')
df.iloc[0:10,:]

,patent_id,raw_assignee
0,3858241,"UNITED STATES OF AMERICA, ARMY"
1,3858242,private
2,3858243,PLYMATIC
3,3858244,private
4,3858245,HAIR AGAIN LTD.
5,3858246,private
6,3858247,private
7,3858248,private
8,3858249,private
9,3858250,private


In [13]:
df = df.loc[df['raw_assignee']!='private',:]
len(df)

4641719

In [18]:
df.reset_index(drop=True,inplace=True)
df.loc[df['patent_id']=='6836905',:]

,patent_id,raw_assignee
2501741,6836905,Sande Hockey Inc.


In [21]:
df = df.iloc[0:2501741,:]
df.to_csv('E:/apps/pid_raw_assignee.csv',index=False)

__raw_assignee -> assignee__

In [30]:
os.chdir("E:/apps/db_table")

df = pd.read_csv('E:/apps/pid_raw_assignee.csv')

ass = pd.read_csv('assignee_raw.csv')
ass['raw_assignee'] = map(lambda x: x.upper(), ass['raw_assignee'])

len(ass)

187081

In [31]:
ass.drop_duplicates(inplace=True)
len(ass)

172725

In [33]:
df = df.merge(ass, how='left', on='raw_assignee')
len(df)

2501741

In [34]:
df.drop('raw_assignee', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
len(df)

2499501

In [37]:
df.dropna(inplace=True)
len(df)

1799943

In [38]:
df.to_csv('patent_assignee_1.csv',index=False)
df1 = pd.read_csv('patent_assignee_2.csv')

df2 = pd.concat([df,df1],ignore_index=True)

len(df2), len(df), len(df1)

(3142592, 1799943, 1342649)

In [39]:
df2.to_csv('patent_assignee.csv',index=False)

### 2. 기업간 adj matrix 제작
- 특정 산업에 해당하는 patent selection (patent_class, pat_citation_byfirm join)  
__=> patent_id, cited, year__  
- patent_id -> assignee -> gvkey (patent_assignee join, assignee_gvkey join)   
__=> gvkey, cited, year__
- cited -> assignee -> gvkey (patent_assignee join, assignee_gvkey join)  
__=> gvkey, gvkey_cited, year__
- year 기준으로 subset, 각각에 대해 adj matrix 제작 (여긴 R 사용)

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('E:/apps/db_table')

__1. patent selection__

In [2]:
df1 = pd.read_csv('pat_citation_byfirm_1.csv')
df2 = pd.read_csv('pat_citation_byfirm_2.csv')

df = pd.concat([df1,df2],ignore_index=True)
len(df), len(df1), len(df2)

(19694038, 6884027, 12810011)

In [3]:
del df1, df2

bio = pd.read_csv('patent_class_BIO.csv')
ee = pd.read_csv('patent_class_EE&IT.csv')

bio = bio.loc[:,['patent_id','year']]
ee = ee.loc[:,['patent_id','year']]

bio = bio.merge(df,how='left',on='patent_id')
ee = ee.merge(df,how='left',on='patent_id')

bio.dropna(how='any',inplace=True)  #drop patents which don't cite apps
ee.dropna(how='any',inplace=True)
print len(bio), len(ee)

3870228 8943754


In [4]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__2. patent_id -> gvkey__

In [7]:
df = pd.read_csv('patent_assignee.csv')

bio = bio.merge(df, how='left', on='patent_id')
ee = ee.merge(df, how='left', on='patent_id')

df = pd.read_csv('assignee_gvkey_BIO.csv')
df1 = pd.read_csv('assignee_gvkey_EE&IT.csv')

bio = bio.merge(df, how='left', on='assignee')
ee = ee.merge(df1, how='left', on='assignee')

bio = bio.loc[:,['gvkey','cited','year']]
ee = ee.loc[:,['gvkey','cited','year']]

len(bio), len(ee)

(3953630, 9056153)

In [8]:
bio.dropna(how='any',inplace=True)  #drop patents which don't have gvkey of assignee
ee.dropna(how='any',inplace=True)

len(bio), len(ee)

(1935285, 4331836)

In [9]:
ee.iloc[0:10,:]

,gvkey,cited,year
22,19349.0,6047552,2005
23,19349.0,5737922,2005
24,19349.0,5467815,2005
30,5047.0,5778676,2005
31,5047.0,5816049,2005
32,5047.0,5865024,2005
33,5047.0,6050082,2005
34,5047.0,6195607,2005
35,5047.0,6449953,2005
36,5047.0,6460340,2005


In [11]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__3. cited -> gvkey_cited__